In [1]:
import pandas as pd

from src.explainer import valid_explainers_dico
from src.io import load_results

pd.options.plotting.backend = "plotly"
from src.scoring import get_score_df, get_eligible_points_df, get_summary_df

In [2]:
result_df = load_results()
# todo make it multi index
score_df = get_score_df(result_df)
eligible_points_df = get_eligible_points_df(result_df)
summary_df = get_summary_df(result_df, score_df, eligible_points_df)
print(summary_df.round(2))

                              time  score  eligible_points  percentage  \
baseline_random               1.00   3.13               18        0.17   
saabas                        0.11   6.04               11        0.55   
tree_shap                     0.09   8.26               11        0.75   
archipelago                   1.11   4.00                5        0.80   
shapley_taylor_interaction    9.03   3.29                5        0.66   
shap_interaction            169.72   1.02                5        0.20   
anova                        49.01   1.13                5        0.23   
sage                        103.59   5.02                7        0.72   
lime                        825.85  10.66               13        0.82   
kernel_shap                  50.43  10.13               11        0.92   
maple                       966.39   9.23               13        0.71   

                            eligible_tests  time_per_test  
baseline_random                         12         

In [3]:
result_df

,cough_and_fever,cough_and_fever_10_90,a_and_b_or_c,x0_plus_x1_distrib_non_uniform_stat_dep,x0_plus_x1_distrib_uniform_stat_dep,x0_plus_x1_distrib_non_uniform_stat_indep,detect_interaction0,detect_interaction1,detect_interaction2,detect_interaction3,detect_interaction4,mnist
baseline_random,{'score': {'importance_symmetric': 0.233156890...,{'score': {'is_cough_more_important_than_fever...,{'score': {'importance_x0_more_important': 0.0...,"{'score': {'importance_symmetric': 0.0, 'is_at...",{'score': {'importance_symmetric': 0.969026384...,{'score': {'importance_symmetric': 0.753531215...,"{'score': {'interaction_detection': 0.0}, 'tim...","{'score': {'interaction_detection': 0.0}, 'tim...",{'score': {'interaction_detection': 0.02701197...,{'score': {'interaction_detection': 0.02701197...,"{'score': {'interaction_detection': 0.0}, 'tim...","{'score': {'importance_dummy': 0.0, 'attributi..."
saabas,"{'score': {'importance_symmetric': 1.0, 'is_at...",{'score': {'is_cough_more_important_than_fever...,{'score': {'importance_x0_more_important': 1.0...,{'score': {'importance_symmetric': 0.616017222...,{'score': {'importance_symmetric': 0.600088864...,{'score': {'importance_symmetric': 0.999378770...,{'Last_updated': '2022-03-26 12:23:48.912916'},{'Last_updated': '2022-03-26 12:23:48.912916'},{'Last_updated': '2022-03-26 12:23:48.913913'},{'Last_updated': '2022-03-26 12:23:48.914910'},{'Last_updated': '2022-03-26 12:23:48.914910'},"{'score': {'importance_dummy': None, 'attribut..."
tree_shap,{'score': {'importance_symmetric': 0.200000032...,{'score': {'is_cough_more_important_than_fever...,{'score': {'importance_x0_more_important': 1.0...,{'score': {'importance_symmetric': 0.997620593...,{'score': {'importance_symmetric': 0.999991153...,{'score': {'importance_symmetric': 0.999380553...,{'Last_updated': '2022-03-26 12:23:49.246535'},{'Last_updated': '2022-03-26 12:23:49.246535'},{'Last_updated': '2022-03-26 12:23:49.246535'},{'Last_updated': '2022-03-26 12:23:49.247531'},{'Last_updated': '2022-03-26 12:23:49.247531'},"{'score': {'importance_dummy': None, 'attribut..."
archipelago,{'Last_updated': '2022-03-26 12:23:49.279954'},{'Last_updated': '2022-03-26 12:23:49.279954'},{'Last_updated': '2022-03-26 12:23:49.280951'},{'Last_updated': '2022-03-26 12:23:49.280951'},{'Last_updated': '2022-03-26 12:23:49.280951'},{'Last_updated': '2022-03-26 12:23:49.281949'},"{'score': {'interaction_detection': 0.0}, 'tim...","{'score': {'interaction_detection': 1.0}, 'tim...","{'score': {'interaction_detection': 1.0}, 'tim...","{'score': {'interaction_detection': 1.0}, 'tim...","{'score': {'interaction_detection': 1.0}, 'tim...",{'Last_updated': '2022-03-26 12:23:55.676175'}
shapley_taylor_interaction,{'Last_updated': '2022-03-26 12:23:55.676175'},{'Last_updated': '2022-03-26 12:23:55.677172'},{'Last_updated': '2022-03-26 12:23:55.677172'},{'Last_updated': '2022-03-26 12:23:55.677172'},{'Last_updated': '2022-03-26 12:23:55.678170'},{'Last_updated': '2022-03-26 12:23:55.678170'},"{'score': {'interaction_detection': 0.0}, 'tim...","{'score': {'interaction_detection': 1.0}, 'tim...",{'score': {'interaction_detection': 0.90746268...,{'score': {'interaction_detection': 0.52835820...,{'score': {'interaction_detection': 0.85492227...,{'Last_updated': '2022-03-26 12:24:10.403280'}
shap_interaction,{'Last_updated': '2022-03-26 12:24:10.403280'},{'Last_updated': '2022-03-26 12:24:10.403280'},{'Last_updated': '2022-03-26 12:24:10.403280'},{'Last_updated': '2022-03-26 12:24:10.404276'},{'Last_updated': '2022-03-26 12:24:10.404276'},{'Last_updated': '2022-03-26 12:24:10.404276'},"{'score': {'interaction_detection': 0.0}, 'tim...","{'score': {'interaction_detection': 1.0}, 'tim...","{'score': {'interaction_detection': 0.0}, 'tim...","{'score': {'interaction_detection': 0.0}, 'tim...",{'score': {'interaction_detection': 0.01554404...,{'Last_updated': '2022-03-26 12:27:04.937066'}
anova,{'Last_updated': '2022-03-26 12:27:04.937066'},{'Last_updated': '2022-03-26 12:27:04

In [4]:
def _round(x,n=0):
    if x is None:
        return None
    return round(x,n)

test_columns = []
for test_name in result_df.columns:
    for sub_test in result_df[test_name][0]['score'].keys():
        new_col = f'{test_name}__{sub_test}'
        test_columns.append(new_col)
        summary_df[new_col] =  [_round(dico.get('score', {}).get(sub_test, None))
                                                   for idx,dico in result_df[test_name].items()]
    summary_df[test_name + '__time'] =  [_round(dico.get('time', None)) for idx,dico in result_df[test_name].items()]

In [5]:
compare = ['sage', 'kernel_shap']  # , 'tree_shap'
assert all([e_name in valid_explainers_dico.keys() for e_name in compare])

In [6]:
df = summary_df.loc[compare].transpose().rename(index={'eligible_points':'_eligible_points',
                                                       'percentage':'_percentage',
                                                       'score':'_score',
                                                       'time':'_time',})
df.round(2)

,sage,kernel_shap
_time,103.59,50.43
_score,5.02,10.13
_eligible_points,7.00,11.00
_percentage,0.72,0.92
eligible_tests,7.00,6.00
time_per_test,14.80,8.40
cough_and_fever__importance_symmetric,0.00,1.00
cough_and_fever__is_attribution_symmetric,NaN,1.00
cough_and_fever__time,0.00,9.00
cough_and_fever_10_90__is_cough_more_important_than_fever,1.00,1.00


In [7]:
_common_rows = set(df.loc[df.notna().apply(all,axis=1)].index)
common_tests = set(test_columns).intersection(_common_rows)

score_common_tests = df.loc[common_tests].sum(axis=0).rename('__score_common_tests')
df.loc['__score_common_tests'] = list(score_common_tests)

In [8]:
list_common_rows = list(_common_rows) + ['__score_common_tests']

In [9]:
df_common = df.loc[list_common_rows].sort_index()
df_common.round(2)

,sage,kernel_shap
__score_common_tests,5.00,6.00
_eligible_points,7.00,11.00
_percentage,0.72,0.92
_score,5.02,10.13
_time,103.59,50.43
a_and_b_or_c__importance_x0_more_important,1.00,1.00
a_and_b_or_c__time,0.00,25.00
cough_and_fever_10_90__is_cough_more_important_than_fever,1.00,1.00
cough_and_fever_10_90__time,0.00,8.00
cough_and_fever__importance_symmetric,0.00,1.00


In [10]:
df_common.loc[common_tests]

,sage,kernel_shap
x0_plus_x1_distrib_uniform_stat_dep__importance_symmetric,1.0,1.0
a_and_b_or_c__importance_x0_more_important,1.0,1.0
x0_plus_x1_distrib_non_uniform_stat_indep__importance_symmetric,1.0,1.0
cough_and_fever__importance_symmetric,0.0,1.0
cough_and_fever_10_90__is_cough_more_important_than_fever,1.0,1.0
x0_plus_x1_distrib_non_uniform_stat_dep__importance_symmetric,1.0,1.0


In [11]:
# todo https://www.quora.com/How-do-I-highlight-a-row-in-Pandas-DataFrame
# todo score sur les tests en commun